[Convex.jl documentation](http://convexjl.readthedocs.org/en/latest/operations.html)

In [1]:
using Convex, SCS
using Gadfly

This notebook implements [Linearly Constrained Market Makers](http://www.cs.princeton.edu/~mdudik/DudikEtAl13.pdf) (LCMMs) aka [Constraint-Generation based Market Makers](http://www.cs.princeton.edu/~mdudik/DudikLaPe12.pdf) (CGMMs), and is a step toward [decreasing utility market makers](http://arxiv.org/abs/1407.8161) that eliminate [just-in-time arbitrage](http://www.probabilityandfinance.com/GTP2014/Slides/Frongillo.pdf) and enable "implicit submarket closing".

Note that in the CGMM paper (Dudik et. al., 2012), the sections 3.3 "Projected gradient-descent update" and 3.4 "Coordinate-descent update for sums of LMSR markets" describe convex optimization methods based on gradient/coordinate-descent. For a comparison of these methods to the Splitting Conic Solver (SCS) used here, which is based on ADMM (alternating direction method of multipliers), see the answers to [this quora question](http://www.quora.com/Convex-Optimization/Whats-the-advantage-of-alternating-direction-method-of-multipliers-ADMM-and-whats-the-use-case-for-this-type-of-method-compared-against-classic-gradient-descent-or-conjugate-gradient-descent-method).

An Linearly Constrained Market Maker (LCMM) market structure, or "block structure", is specified by partitioning logically-related securities into independently-priced "blocks" or groups, which are treated as separate "submarkets". The logical relations then become constraints on the state prices within and across these submarkets.

We start by specifying a block structure for a prediction market on Norway's performance in the Olympic games (see [Section 4.1 Linearly Constrainted Markets, Example 7](http://arxiv.org/abs/1407.8161)).  We choose a block structure that enables two types of bets: (1) bets on individual games (e.g. Norway wins gold in swimming), and (2) bets on how many gold medals are won in total (e.g. Norway wins 5 gold medals).

We begin with an example world where there are three olympic games. This block structure has four submarkets with seven securities in total (i.e. K = 2n + 1, and n = 3). We represent the prices of these securities with the vector K:
  * K1 : Norway wins gold in men's weightlifting
  * K2 : Norway wins gold in men's swimming
  * K3 : Norway wins gold in men's curling
  * K4 : Norway wins a total of 0 gold medals
  * K5 : Norway wins a total of 1 gold medal
  * K6 : Norway wins a total of 2 gold medals
  * K7 : Norway wins a total of 3 gold medals

The block structure for this example is: { {K1}, {K2}, {K3}, {K4, K5, K6, K7} }. That is, there is one submarket each for K1, K2, and K3, and a submarket containing the four mutually-exclusive events K4-K7.

####Note:
Ended up with a slightly different block structure than the one specified in Section 4.1  Example 7 of Dudik-2014. Instead of a single security on each game (win), there are two securities (win and lose). Thus, we have the securities:

* K1 : Norway wins gold in men's weightlifting
* K2 : Norway does not win gold in weightlifting
* K3 : Norway wins gold in men's swimming
* K4 : Norway does not win gold in swimming
* K5 : Norway wins gold in men's curling
* K6 : Norway does not win gold in curling
* K7 : Norway wins a total of 0 gold medals
* K8 : Norway wins a total of 1 gold medal
* K9 : Norway wins a total of 2 gold medals
* K10 : Norway wins a total of 3 gold medals

and the block structure: { {K1,K2}, {K3,K4}, {K5,K6}, {K7, K8, K9, K10} }


### Symbols and variable names

* **K** : vector of security prices in the original security space (i.e. independent submarkets with unconstrained prices).

* **q** : market maker inventory vector, with dimensionality K

**A**<sup>T</sup>**µ** >= **b**

> Letting a<sub>*m*</sub> denote the *m*th column of A, arbitrage opportunities open up if the price of the bundle a<sub>*m*</sub> falls below b<sub>*m*</sub>. For any **η** ∈ R<sup>M</sup><sub>+</sub>, the bundle **Aη** presents an arbitrage opportunity if priced below **b** · **η**.


* **A** : matrix of constraints on security prices, with dimensionality KxM (K rows and M columns). one row per security, and one column per constraint.

* **µ** : vector representing market maker's beliefs about event probabilities. same dimensionality as K.

* **b** : column vector of constants (constraint equations are generated by the logical relations between submarkets). has dimensionality M (number of constraints).

C(**q**) = inf<sub>η∈R<sup>M</sup><sub>+</sub></sub>[C<sub>⊕</sub>(**q** + **Aη**) - **b**·**η**]

* objective function chooses **η** to minimize the difference between the direct-sum cost of a bundle **q** and arbitrage payoffs across submarkets

* **η** : the arbitrage bundle, or "bundle of bundles". has dimensionality of M extended market maker state

* C<sub>⊕</sub>(**q**) : direct sum of cost functions (i.e. one cost function per submarket).


C<sup></sup>(**q**, **η**) = C<sub>⊕</sub>(**q** + **Aη**) - **b**·**η**

* C(**q**, **η**) : extended cost function




In [2]:
# matrix A contains the logical constraints between prices in the independent submarket blocks
# each row in the matrix is a bundle (or constraint), each column is a security

A_transpose = [
     1 1 0 0 0 0 0 0 0 0; # submarket for first game, (weighlifting) (two mutually-exclusive options, wins or loses)
     0 0 1 1 0 0 0 0 0 0; # submarket for second game (swimming)
     0 0 0 0 1 1 0 0 0 0; # submarket for third game (curling)
     0 0 0 0 0 0 1 1 1 1; # submarket for total count (four mutually-exclusive options)
     1 0 1 0 1 0 1 0 0 0; # guaranteed to pay out 1 (either zero medals, or >= 1 medal)
     0 1 0 1 0 1 0 1 1 1; # guaranteed to pay out 1 (either >= 1 medal, or loses every event)
    -1 0 -1 0 -1 0 0 1 2 3; # sum of coefficients equals zero. if three gold medals are won, then -3 + 3 == 0
    1 0 1 0 1 0 0 -1 -2 -3; # equality converted to two inequalities (i.e. +h >= 0 and -h >= 0)
    ];


A = A_transpose'

b = [1; 1; 1; 1; 1; 1; 0; 0;] # 8 constraints

8-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 0
 0

### Test some trade sequences using LCMM with medal counts submarket structure

In [3]:
# B_param is the liquidity parameter
# chosen arbitrarily to work okay with the component amounts in q_vec_one and q_vec_two 
B_param = 25

# η is the "bundle of bundles" (i.e. the market maker's internal vector of arbitrage bundles)
η_init = Variable(8, Positive());
η_one = Variable(8, Positive());
η_two = Variable(8, Positive());

# price propagation enabled, as arbitrage bundles can be greater than zero
# DISABLES arbitrage opportunities
# * arbitrage opportunities WILL NOT exist between submarkets
# * market maker purchases arbitrage bundles before traders
constraint_init = [η_init >= 0]
constraint_one = [η_one >= 0]
constraint_two = [η_two >= 0]


# these constraints will set arbitrage bundles to zero, so prices can't propagate
# used for testing (compare to regular LMSR cost functions)
# ENABLES arbitrage opportunities
# * arbitrage opportunities WILL exist between submarkets
constraint_arb_init = η_init == 0
constraint_arb_one = η_one == 0
constraint_arb_two = η_two == 0


# the cost of the q_zero bundle is the subsidy to create the market
q_zero = [0,0,0,0,0,0,0,0,0,0] 



# these vectors test if prices propagate from submarket to submarket.
# trade_two would cost a (uniform-prior) 0.25 if they were independent.
# but since prices propagate, it costs 0.68
#q_vec_one = [0,100,0,100,0,100,0,0,0,0]
#q_vec_two = [0,100,0,100,0,100,1,0,0,0] 



# q_vec_one sets the most likely outcome to be Norway wins 0 gold medals
# so q_vec_two (on Norway wins a gold medal in weightlifting)
#   should cost less than 0.5 (the uniform-prior)
# with arbitrage enabled, prices don't propagate and q_vec_two costs 0.5
# with arbitrage disabled, prices propagate and q_vec_two costs 0.0486
#q_vec_one = [0,0,0,0,0,0,300,0,0,0]
#q_vec_two = [1,0,0,0,0,0,300,0,0,0] 


# q_vec_one sets the most likely outcome for each independent game to be "does not win gold"
# q_vec_two buys one share on "norway wins 0 gold" and one share each on "wins gold" for each game
# q_vec_two has a guaranteed payoff of at least 1
#   so if prices do not propagate it might be an arbitrage opportunity
# q_vec_two with arbitrage disabled costs 0.999 (no arbitrage opportuntity)
# q_vec_two with arbitrage enabled costs 0.2537 (the 0.25 uniform price on the total count submarket)
#   (plus the 0.0037.. cost of "wins gold" on all three games)
q_vec_one = [0,350,0,350,0,350,0,0,0,0]
q_vec_two = [1,350,1,350,1,350,1,0,0,0] 



initial_obj = (B_param*logsumexp( (q_zero[1:2] + (A*η_init)[1:2])/B_param )
              + B_param*logsumexp( (q_zero[3:4] + (A*η_init)[3:4])/B_param )
              + B_param*logsumexp( (q_zero[5:6] + (A*η_init)[5:6])/B_param )
              + B_param*logsumexp( (q_zero[7:10] + (A*η_init)[7:10])/B_param ) - (b'*η_init))


vec_one_obj = (B_param*logsumexp( (q_vec_one[1:2] + (A*η_one)[1:2])/B_param )
              + B_param*logsumexp( (q_vec_one[3:4] + (A*η_one)[3:4])/B_param )
              + B_param*logsumexp( (q_vec_one[5:6] + (A*η_one)[5:6])/B_param )
              + B_param*logsumexp( (q_vec_one[7:10] + (A*η_one)[7:10])/B_param ) - (b'*η_one))


vec_two_obj = (B_param*logsumexp( (q_vec_two[1:2] + (A*η_two)[1:2])/B_param )
              + B_param*logsumexp( (q_vec_two[3:4] + (A*η_two)[3:4])/B_param )
              + B_param*logsumexp( (q_vec_two[5:6] + (A*η_two)[5:6])/B_param )
              + B_param*logsumexp( (q_vec_two[7:10] + (A*η_two)[7:10])/B_param ) - (b'*η_two))



initial_cost = minimize(initial_obj, constraint_init)
q_vec_one_cost = minimize(vec_one_obj, constraint_one)
q_vec_two_cost = minimize(vec_two_obj, constraint_two)

initial_arbable_cost = minimize(initial_obj, constraint_arb_init)
q_vec_one_cost_arbable = minimize(vec_one_obj, constraint_arb_one)
q_vec_two_cost_arbable = minimize(vec_two_obj, constraint_arb_two)


Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: ConvexVexity()

subject to
Constraint:
== constraint
lhs: Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
rhs: 0
vexity: AffineVexity()
current status: not yet solved

In [4]:
solve!(initial_cost, SCSSolver(verbose=3))

----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 1
	linear vars: 20
	exp vars: 30, dual exp vars: 0
Setup time: 1.18e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  9.50e-04 
   100| 3.12e-03  5.48e-01  5.31e-03  8.66e+01  8.75e+01  1.35e-14  2.71e-02 
   200| 4.53e-05  3.96e-03  5.25e-05  8.66e+01  8.66e+01  4.51e-15  5.45e-02 
   280| 1.93e-06  9.32e-05  1.7

In [5]:
initial_cost.status

:Optimal

In [6]:
initial_cost.optval # subsidy to create or initialize market

86.64317923612931

In [7]:
η_init

Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
value: 8x1 Array{Float64,2}:
 8.69185e-7 
 8.69185e-7 
 8.69185e-7 
 4.40509e-6 
 9.46841e-10
 4.22506e-10
 0.000544808
 0.0010786  

In [8]:
solve!(q_vec_one_cost, SCSSolver(verbose=3))

----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 1
	linear vars: 20
	exp vars: 30, dual exp vars: 0
Setup time: 1.52e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan       inf       inf       inf  2.85e-04 
   100| 5.47e-03  5.63e-01  7.29e-06  1.07e+03  1.07e+03  5.23e-14  4.73e-02 
   200| 6.17e-03  4.20e-01  2.72e-06  1.06e+03  1.06e+03  0.00e+00  9.87e-02 
   300| 5.65e-03  3.20e-01  3.3

In [9]:
q_vec_one_cost.optval

1050.0803250973631

In [10]:
# bundle of bundles (each row is a quantity of one of the constraint bundles in the A matrix)
η_one

Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
value: 8x1 Array{Float64,2}:
  0.000329687
  0.000329687
  0.000329687
  4.08538e-5 
 63.7263     
  4.39947e-8 
  3.66623e-8 
 97.6806     

In [11]:
# arbitrage bundle in the original security space
A*η_one.value

10x1 Array{Float64,2}:
  161.407      
    0.000329731
  161.407      
    0.000329731
  161.407      
    0.000329731
   63.7263     
  -97.6806     
 -195.361      
 -293.042      

In [12]:
trade_one_cost = q_vec_one_cost.optval - initial_cost.optval

963.4371458612338

In [13]:
solve!(q_vec_two_cost, SCSSolver(verbose=3))

----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 1
	linear vars: 20
	exp vars: 30, dual exp vars: 0
Setup time: 1.23e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan       inf       inf       inf  5.42e-04 
   100| 5.36e-03  5.55e-01  2.33e-05  1.07e+03  1.07e+03  0.00e+00  5.97e-02 
   200| 6.05e-03  4.15e-01  1.95e-05  1.06e+03  1.06e+03  0.00e+00  1.14e-01 
   300| 5.56e-03  3.16e-01  1.4

In [14]:
q_vec_two_cost.optval

1051.0803794041922

In [15]:
η_two

Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
value: 8x1 Array{Float64,2}:
  0.000327812
  0.000327812
  0.000327812
  4.10394e-5 
 63.3105     
  1.66283e-8 
 -4.39826e-9 
 97.1        

In [16]:
A*η_two.value

10x1 Array{Float64,2}:
  160.411      
    0.000327829
  160.411      
    0.000327829
  160.411      
    0.000327829
   63.3105     
  -97.1        
 -194.2        
 -291.3        

In [17]:
# cost is 1.0 because market-maker "front-runs" the arbitrage opportunity by buying η_two
trade_two_cost = q_vec_two_cost.optval - q_vec_one_cost.optval

1.0000543068290426

### Now test the same sequence of bundles without removing arbitrage opportunities

In [18]:
solve!(initial_arbable_cost, SCSSolver(verbose=3))

----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 9
	linear vars: 12
	exp vars: 30, dual exp vars: 0
Setup time: 1.38e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.17e-04 
   100| 3.12e-03  5.48e-01  5.31e-03  8.66e+01  8.75e+01  1.80e-14  2.60e-02 
   200| 4.52e-05  3.96e-03  5.24e-05  8.66e+01  8.66e+01  0.00e+00  5.27e-02 
   280| 1.87e-06  9.10e-05  1.7

In [19]:
initial_arbable_cost.status

:Optimal

In [20]:
initial_arbable_cost.optval

86.64317535617148

In [21]:
solve!(q_vec_one_cost_arbable, SCSSolver(verbose=3))


----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 9
	linear vars: 12
	exp vars: 30, dual exp vars: 0
Setup time: 1.44e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan       inf       inf       inf  2.82e-04 
    60| 6.60e-06  2.70e-05  3.88e-08  1.08e+03  1.08e+03  4.92e-14  2.62e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Total sol

In [22]:
q_vec_one_cost_arbable.optval

1084.6575383996378

In [23]:
η_one

Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
value: 8x1 Array{Float64,2}:
  3.37663e-11
  3.37663e-11
  3.37663e-11
  6.64101e-11
 -3.82462e-7 
 -9.08304e-7 
 -8.9174e-7  
 -3.89386e-7 

In [24]:
trade_one_cost_arbable = q_vec_one_cost_arbable.optval - initial_arbable_cost.optval

998.0143630434663

In [25]:
solve!(q_vec_two_cost_arbable, SCSSolver(verbose=3))


----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 89
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 23, constraints m = 51
Cones:	primal zero / dual free vars: 9
	linear vars: 12
	exp vars: 30, dual exp vars: 0
Setup time: 1.17e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan       inf       inf       inf  2.55e-04 
    60| 6.04e-06  2.67e-05  4.55e-08  1.08e+03  1.08e+03  4.91e-14  3.32e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Total sol

In [26]:
q_vec_two_cost_arbable.optval

1084.9113051184097

In [27]:
# arbitrage opportunity because cost is 0.2537.. for a guaranteed payoff of 1.0
trade_two_cost_arbable = q_vec_two_cost_arbable.optval - q_vec_one_cost_arbable.optval

0.2537667187718853

In [28]:
η_two

Variable of
size: (8, 1)
sign: Positive()
vexity: AffineVexity()
value: 8x1 Array{Float64,2}:
  3.49672e-11
  3.49672e-11
  3.49672e-11
  6.39371e-11
 -3.73903e-7 
 -9.09838e-7 
 -8.91481e-7 
 -4.03853e-7 

### Todo:
* chart the LCMM prices of each outcome
* compare prices on the independent submarkets to simple LMSR prices